<h1>Imports and Data</h1>

In [ ]:
import numpy as np

from numpy.linalg import inv, pinv

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

<h1>Data</h1>

In [ ]:
w_1 = 8.0 #w_1
w_2p = 10.0 #w_2'

m = w_1/w_2p
c = 0.1

w_0 = w_2p*c
w_2 = -w_2p

w_true = np.array([w_0, w_1, w_2])

N = 100

x = np.random.rand(3,N)
x[0,:] = 1

print x.shape #should print (3L, 10L) [windows] or (3,10) [linux] showing 3x10 matrix
print w_true.shape #should pring (3L,) [windows] or (3,) [linux] showing 3x1 matrix
#note that it is row vector instead or column

#now let us compute the labels
y_true = (w_true.dot(x)>0)*1.0+(w_true.dot(x)<=0)*-1.0

marker = {}
marker[-1.0] = '+b'
marker[1.0] = '*r'

for i in range(N):
    plt.plot(x[1,i],x[2,i],marker[y_true[i]])

x_1__ = np.arange(0,1.0,0.001)
x_2__ = m*x_1__+c #the line that separates the classes


plt.plot(x_1__,x_2__,'-k')
plt.xlim([0,1.0])
plt.ylim([0,1.0])

plt.show()

#you should see a perfectly separed data

<h1>Support Vector Machine</h1>

In [ ]:
from numpy import linalg
import cvxopt
from cvxopt import matrix
import cvxopt.solvers

X = x[1:3,:]
P = np.zeros((N,N),dtype=np.double)
for i in range(N):
    for j in range(N):
        P[i,j] = y_true[i]*y_true[j]*X[:,i].dot(X[:,j])

P = matrix(P)
q = matrix(np.ones(N)*-1.0)

G = matrix(np.diag(np.ones(N))*-1.0)
h = matrix(np.zeros(N))

A = matrix(y_true.reshape(1,N))
b = matrix(0.0)

solution = cvxopt.solvers.qp(P, q, G, h, A, b)

a = np.ravel(solution['x'])

sv = a>0
a_sv = a[sv]
X_sv = X[:,sv]
y_sv = y_true[sv]


w_svm = np.sum(a_sv*y_sv*X_sv,axis=1)

YP = w_svm.dot(X_sv)

b = -(np.min(YP[y_sv>0])+np.max(YP[y_sv<=0]))/2.0

#for i in range(len(a)):
#    if a[i]>10^-5:
#        b = 1/y_true[i] - w_svm.dot(X[:,i])
#        break

b = np.array(b).reshape(1,1)
w_svm = w_svm.reshape(1,len(w_svm))
w_est = np.concatenate((b,w_svm),axis=1)
print w_est

<h1>Performance</h1>

In [ ]:
w_est2 = w_est[0]
m_est = w_est2[1]/np.abs(w_est2[2])
c_est = w_est2[0]/np.abs(w_est2[2])

print m_est, c_est

x_2_est = m_est*x_1__ + c_est

for i in range(N):
    plt.plot(x[1,i],x[2,i],marker[y_true[i]])

plt.plot(x_1__,x_2__, 'k')
plt.plot(x_1__,x_2_est,'r')
plt.xlim([0,1.0])
plt.ylim([0,1.0])
plt.show()